# Introduction
This notebook contains a basic trend-following breakout strategy that can be used as a starting point to build and test your own strategy upon.

In [1]:
%use roboquant(3.0.0-SNAPSHOT)
Welcome()

roboquant 3.0.0-SNAPSHOT 
 build: 2025-05-06T09:22:02Z 
 home: /Users/peter/.roboquant 
 os: Mac OS X 15.4.1 
 jvm: OpenJDK 64-Bit Server VM 17.0.15 
 memory: 3256 MB 
 cpu cores: 8

Below is the barebone minumum to run a back test. It uses a feed and strategy that come out of the box with roboquant:

- A feed containing the prices for the 25 most active US stocks from the past 4 years (S&P 25)
- The EMA Crossover strategy with its default settings.

In [2]:
val feed = AvroFeed.sp25()
val strategy = EMACrossover()
run(feed, strategy)

last update  : 2024-12-31T21:00:00Z
cash         : USD 940,894.94
buying Power : USD 940,894.94
equity       : USD 1,134,824.62
positions    : 0@JPM, 0@WMT, 95@XOM, 78@NVDA, 0@MA, 24@BRK.B, 41@V, 0@META, 63@ABBV, 0@CVX, 48@AAPL, 54@AMZN, 17@NFLX, 68@JNJ, 0@HD, 19@UNH, 61@AVGO, 66@GOOG, 67@GOOGL, 0@MSFT, 46@TSLA, 0@PG, 14@LLY, 0@MRK, 12@COST
open orders  : -54@AMZN, -78@NVDA, -17@NFLX

Typically however you'll want to develop your own strategy. Below a simple trend following strategy that implements the following rules:

- If the last price is the maximum price over the past <period> days, generate a BUY rating (1.0)
- If the last price is the minimum price over the past <period> days, generate a SELL rating (-1.0)
- In all other cases don't generate a rating

In [3]:
/**
 * Basic breakout strategy.
 *
 * @param period the historic period to use for checking this is a breakout
 */
class MyStrategy(period: Int) : HistoricPriceStrategy(period) {

    /**
     * This method will be called by the superclass for every asset in the feed once
     * there is enough data collected for that asset.
     */
    override fun generateRating(data: DoubleArray): Double? {
        return when (data.last()) {
            data.max() -> BUY
            data.min() -> SELL
            else -> null
        }
    }

}

In [4]:
// Create an instance of MyStrategy with 26 days worth of history
val strategy = MyStrategy(period = 26)

// Define the metric(s) you want to monitor
val journal = MemoryJournal(AccountMetric())

In [5]:
// Run the back test
run(feed, strategy, journal)

last update  : 2024-12-31T21:00:00Z
cash         : USD 938,171.67
buying Power : USD 938,171.67
equity       : USD 1,136,056.27
positions    : 0@MSFT, 110@MRK, 0@NVDA, 0@AVGO, 66@GOOG, 78@CVX, 67@GOOGL, 41@TSLA, 0@PG, 60@AAPL, 26@HD, 11@COST, 147@WMT, 0@XOM, 58@AMZN, 0@BRK.B, 15@NFLX, 0@JNJ, 0@UNH, 51@JPM, 0@LLY, 18@META, 24@MA, 65@ABBV, 39@V
open orders  : 79@JNJ, -11@COST

In [6]:
// Plot the equity curve
val equity = journal.getMetric("account.equity")
TimeSeriesChart(equity)

<!-- roboquant TimeSeriesChart -->

A very simple walkforward test over 1-year periods and at the end of the run printing the total equity hold in the account

In [7]:
feed.timeframe.split(1.years).forEach { 
    val account = run(feed, EMACrossover(), timeframe=it)
    println("$it => ${account.equity()}")
}

[2021-01-04T21:00:00Z - 2022-01-04T21:00:00Z> => USD 1,042,048.59
[2022-01-04T21:00:00Z - 2023-01-04T21:00:00Z> => USD 984,506.12
[2023-01-04T21:00:00Z - 2024-01-04T21:00:00Z> => USD 1,031,151.48
[2024-01-04T21:00:00Z - 2024-12-31T21:00:00Z] => USD 1,037,149.01
